In [1]:
import numpy as np

In [2]:
import trading_vix_env

In [3]:
local_env = trading_vix_env.trading_vix_env()

/home/zuhao/anaconda2/envs/p37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
state = local_env.reset()

In [5]:
type(state)

numpy.ndarray

In [6]:
state.shape

(6,)

In [7]:
state

array([ 14.82      ,   5.44956836,   1.        ,  62.        ,
       103.        ,   0.        ])

In [8]:
np.reshape(state,(-1,))

array([ 14.82      ,   5.44956836,   1.        ,  62.        ,
       103.        ,   0.        ])

In [9]:
import gym
env = gym.make('Pendulum-v0')

In [10]:
gym_state = env.reset()

In [11]:
print(type(gym_state))
print(gym_state.shape)

<class 'numpy.ndarray'>
(3,)


In [12]:
state

array([ 14.82      ,   5.44956836,   1.        ,  62.        ,
       103.        ,   0.        ])

In [13]:
import numpy as np
np.reshape(state,(-1,)).shape

(6,)

In [14]:
from stable_baselines3.common.env_checker import check_env

In [15]:
vix_env = trading_vix_env.trading_vix_env()

/home/zuhao/anaconda2/envs/p37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [16]:
check_env(vix_env, warn=True)

/home/zuhao/anaconda2/envs/p37/lib/python3.7/site-packages/stable_baselines3/common/env_checker.py:232: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [22]:
#training

In [23]:
vix_env = trading_vix_env.trading_vix_env()

In [24]:
vix_env.action_space

Box(0.0, 1.0, (1,), float32)

In [ ]:


def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env

    return _init

In [ ]:
from stable_baselines3.common.env_util import make_vec_env,SubprocVecEnv

num_cpu = 10
train_env = env = SubprocVecEnv([make_env(vix_env, i) for i in range(num_cpu)], start_method='spawn')

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env,SubprocVecEnv

env = make_vec_env(trading_vix_env.trading_vix_env,n_envs = 10)
model = A2C('MlpPolicy', env, verbose=1,n_steps = 10*201)
model.learn(total_timesteps=1000000000)


In [ ]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    return _init

def main():
    env_id = trading_vix_env.trading_vix_env
    num_cpu = 20  # Number of processes to use
    # Create the vectorized environment
    env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

    model = A2C('MlpPolicy', env, verbose=1,n_steps = 201)
    model.learn(total_timesteps=25000)

    obs = env.reset()
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        env.render()

if __name__ == '__main__':
    main()

In [ ]:
import gym
import numpy as np


from stable_baselines3.common.vec_env import SubprocVecEnv


def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    return _init

def main():
    env_id = "CartPole-v1"
    num_cpu = 4  # Number of processes to use
    # Create the vectorized environment
    env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

    model = A2C('MlpPolicy', env, verbose=1)
    model.learn(total_timesteps=25000)

    obs = env.reset()
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        env.render()

if __name__ == '__main__':
    main()